<center><h1 style="color:#1a1a1a;
                    font-size:3em">
        NHANES - MTech Project 
        </h1> 
        <h2 style="color:#1a1a1a;
                    font-size:2em">
        Feature Selection - Previous Studies fields - Correlation - Data collection (Lab)
       </h2>
       <h3 style="color:#1a1a1a;
                    font-size:2em">
        Prakash Easow Thomas - 2022DA04285 - June 2024
       </h3>
</center>

# 1. Importing Libraries and Reading the Datasets

## 1.1 Importing the Libraries

In [5]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

## 1.2 Reading the NHANES 2018 Datasets

In [7]:
df1lab  = pd.read_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\laboratory.csv')
df2demo = pd.read_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\demographics.csv')
df3diet = pd.read_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\dietary.csv')
df4exam = pd.read_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\examination.csv')
df5ques = pd.read_csv('C:\Prakash\Personal\Studies\Techincal\MTech. Data Science and Engineering\Semester-wise\Semester-I\Study\Sem-I - 0- Python Fundamentals for Data Science (S1-22_DSECLPFDS)\PythonPgm\MTech-Proj03-NHANES-2018\Data\questionnaire.csv')

## 1.3 Variables from the Previous Studies (Excluding lab Variables) + HbA1c

In [9]:
columns_to_select = [
'LBXGLU',	    #Fasting Glucose (mg/dL)
'LBXSGL',   	#Glucose, refrigerated serum (mg/dL)
'LBDHDD',	    #Direct HDL-Cholesterol (mg/dL)
'LBDLDL',   	#LDL-cholesterol (mg/dL)
'LBXTC',    	#Total Cholesterol (mg/dL)
'URXUCR',   	#Creatinine, urine (mg/dL)
'URDACT',   	#Albumin creatinine ratio (mg/g)
'LBXSCR',   	#Creatinine, refrigerated serum (mg/dL)
'LBXSGTSI', 	#Gamma Glutamyl Transferase (GGT) (IU/L)
'LBXHGB',   	#Hemoglobin (g/dL)
'LBXMCHSI', 	#Mean cell hemoglobin (pg)
'LBXMC',    	#Mean cell hemoglobin concentration (g/dL)
'LBXIN',    	#Insulin (uU/mL)
'URXPREG',  	#Urine Pregnancy Result
'LBXTR',    	#Triglyceride (mg/dL)
'LBXSUA',    	#Uric acid (mg/dL)
'BMXBMI',        #	Body Mass Index (kg/m**2)    
'LBXGH'         #Glycohemoglobin (%)

]   

# 2. Merging and Selection of Data - NHANES 2018 Data

## 2.1 Merging Data

In [11]:
common_key = 'SEQN'
# Merge the DataFrames
merged_dfNHAENES2018 = df1lab.merge(df2demo, on=common_key, how='outer')
merged_dfNHAENES2018 = merged_dfNHAENES2018.merge(df3diet, on=common_key, how='outer')
merged_dfNHAENES2018 = merged_dfNHAENES2018.merge(df4exam, on=common_key, how='outer')
merged_dfNHAENES2018 = merged_dfNHAENES2018.merge(df5ques, on=common_key, how='outer')

# Save the merged DataFrame to a new CSV file
merged_dfNHAENES2018.to_csv('Data\Before_Full_Feature_Selection_09_Data.csv', index=False)

print("Files merged successfully into 'Before_Full_Feature_Selection_09_Data.csv'")

Files merged successfully into 'Before_Full_Feature_Selection_09_Data.csv'


In [12]:
merged_dfNHAENES2018.head()

,SEQN,URXUMA,URXUMS,URXUCR,URXCRS,URDACT,WTSSGP2Y,SSAGP,WTSA2YR,URXUAS,...,WHD110,WHD120,WHD130,WHD140,WHQ150,WHQ190,WHQ200,WHQ030M,WHQ500,WHQ520
0,93703.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,93704.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,93705.0,3.2,3.2,24.0,2121.6,13.33,NaN,NaN,NaN,NaN,...,150.0,130.0,63.0,170.0,62.0,2.0,NaN,NaN,NaN,NaN
3,93706.0,39.6,39.6,69.0,6099.6,57.39,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,150.0,17.0,2.0,NaN,NaN,NaN,NaN
4,93707.0,41.6,41.6,209.0,18475.6,19.90,NaN,NaN,24984.393699,5.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,2.0


## 2.2 Selection of Data

In [14]:
valid_columns = pd.Index(columns_to_select).intersection(merged_dfNHAENES2018.columns)
merged_dfNHAENES2018_selected = merged_dfNHAENES2018[columns_to_select]
nan_counts = merged_dfNHAENES2018_selected.isna().sum()
print("\nCount of NaN values in each column:")
print(nan_counts)


Count of NaN values in each column:
LBXGLU      6363
LBXSGL      3353
LBDHDD      2516
LBDLDL      6446
LBXTC       2516
URXUCR      1622
URDACT      1622
LBXSCR      3351
LBXSGTSI    3352
LBXHGB      1726
LBXMCHSI    1726
LBXMC       1726
LBXIN       6429
URXPREG     8198
LBXTR       6420
LBXSUA      3353
BMXBMI      1249
LBXGH       3209
dtype: int64


# 3. Data Preprocessing - NHANES 2018 Data
         - Remove the Non-Numeric Fields 
         - Remove the records where (LBXGH) No HbA1c is not present  
         - Initilized NaN with zeros   

## 3.1  Removing the Non-Numeric Data

In [16]:
def clean_data(df):
    for column in df.columns:
        # Check if the column contains byte-strings
        if df[column].dtype == object:
            # Decode byte-strings to regular strings
            df[column] = df[column].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
        # Convert non-numeric values to NaN for numeric columns
        df[column] = pd.to_numeric(df[column], errors='ignore')
    return df

In [17]:
merged_dfNHAENES2018_Cleaned =clean_data(merged_dfNHAENES2018_selected)
print(merged_dfNHAENES2018_Cleaned)

      LBXGLU  LBXSGL  LBDHDD  LBDLDL  LBXTC  URXUCR  URDACT  LBXSCR  LBXSGTSI  \
0        NaN     NaN     NaN     NaN    NaN     NaN     NaN     NaN       NaN   
1        NaN     NaN     NaN     NaN    NaN     NaN     NaN     NaN       NaN   
2        NaN    85.0    60.0     NaN  157.0    24.0   13.33    0.92      21.0   
3        NaN    94.0    47.0     NaN  148.0    69.0   57.39    0.81      16.0   
4        NaN   115.0    68.0     NaN  189.0   209.0   19.90    0.64       9.0   
...      ...     ...     ...     ...    ...     ...     ...     ...       ...   
9249     NaN     NaN     NaN     NaN    NaN     NaN     NaN     NaN       NaN   
9250     NaN     NaN     NaN     NaN    NaN     NaN     NaN     NaN       NaN   
9251     NaN     NaN     NaN     NaN    NaN     NaN     NaN     NaN       NaN   
9252     NaN     NaN     NaN     NaN    NaN     NaN     NaN     NaN       NaN   
9253     NaN     NaN     NaN     NaN    NaN     NaN     NaN     NaN       NaN   

      LBXHGB  LBXMCHSI  LBX

C:\Users\praka\AppData\Local\Temp\ipykernel_38364\3940003188.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = pd.to_numeric(df[column], errors='ignore')


## 3.2  Removing the records where HbA1c(LBXGH) is not present

In [19]:
merged_dfNHAENES2018_Cleaned_numeric = merged_dfNHAENES2018_Cleaned.select_dtypes(include=[int, float])
column_to_check_HbA1c ='LBXGH'
merged_dfNHAENES2018_Cleaned_numeric_HBV =merged_dfNHAENES2018_Cleaned_numeric.dropna(subset=[column_to_check_HbA1c])
column_to_check_BMI = 'BMXBMI'
merged_dfNHAENES2018_Cleaned_numeric_HBV =merged_dfNHAENES2018_Cleaned_numeric_HBV.dropna(subset=[column_to_check_BMI])

merged_dfNHAENES2018_Cleaned_numeric_HBV =merged_dfNHAENES2018_Cleaned_numeric_HBV.reset_index(drop=True)

#dataset_for_ML = dataset_for_ML.reset_index(drop=True)
print(merged_dfNHAENES2018_Cleaned_numeric_HBV)

      LBXGLU  LBXSGL  LBDHDD  LBDLDL  LBXTC  URXUCR  URDACT  LBXSCR  LBXSGTSI  \
0        NaN    85.0    60.0     NaN  157.0    24.0   13.33    0.92      21.0   
1        NaN    94.0    47.0     NaN  148.0    69.0   57.39    0.81      16.0   
2        NaN   115.0    68.0     NaN  189.0   209.0   19.90    0.64       9.0   
3      122.0   116.0    88.0   109.0  209.0    47.0   19.36    0.58      22.0   
4        NaN    96.0    65.0     NaN  176.0   215.0   12.28    1.32      31.0   
...      ...     ...     ...     ...    ...     ...     ...     ...       ...   
5946   158.0   150.0    60.0    43.0  119.0    51.0   10.39    0.70      18.0   
5947     NaN   101.0    49.0     NaN  182.0   231.0    4.42    0.92      28.0   
5948    94.0    88.0    54.0   108.0  172.0    31.0   10.97    0.69       8.0   
5949     NaN    84.0    34.0     NaN  150.0   177.0   47.01    0.85      16.0   
5950    91.0    90.0    34.0    75.0  163.0    75.0   70.93    0.82      72.0   

      LBXHGB  LBXMCHSI  LBX

In [20]:
nan_counts = merged_dfNHAENES2018_Cleaned_numeric_HBV.isna().sum()
print("\nCount of NaN values in each column:")
print(nan_counts)


Count of NaN values in each column:
LBXGLU      3113
LBXSGL       147
LBDHDD       115
LBDLDL      3194
LBXTC        115
URXUCR        79
URDACT        79
LBXSCR       145
LBXSGTSI     146
LBXHGB         3
LBXMCHSI       3
LBXMC          3
LBXIN       3177
URXPREG     4956
LBXTR       3168
LBXSUA       147
BMXBMI         0
LBXGH          0
dtype: int64


In [21]:
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXHGB'].isna(), 'LBXHGB'] = 14
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXMCHSI'].isna(), 'LBXMCHSI'] = 29
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXMC'].isna(), 'LBXMC'] = 34
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXIN'].isna(), 'LBXIN'] = 8
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['URXPREG'].isna(), 'URXPREG'] = 2
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXTR'].isna(), 'LBXTR'] = 90
merged_dfNHAENES2018_Cleaned_numeric_HBV.loc[merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXSUA'].isna(), 'LBXSUA'] = 6

merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXSGL'].fillna(merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXSGL'].min(), inplace=True)
merged_dfNHAENES2018_Cleaned_numeric_HBV['LBDHDD'].fillna(merged_dfNHAENES2018_Cleaned_numeric_HBV['LBDHDD'].min(), inplace=True)
merged_dfNHAENES2018_Cleaned_numeric_HBV['LBDLDL'].fillna(merged_dfNHAENES2018_Cleaned_numeric_HBV['LBDLDL'].min(), inplace=True)
merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXTC'].fillna(merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXTC'].min(), inplace=True)
merged_dfNHAENES2018_Cleaned_numeric_HBV['URXUCR'].fillna(merged_dfNHAENES2018_Cleaned_numeric_HBV['URXUCR'].min(), inplace=True)
merged_dfNHAENES2018_Cleaned_numeric_HBV['URDACT'].fillna(merged_dfNHAENES2018_Cleaned_numeric_HBV['URDACT'].min(), inplace=True)
merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXSCR'].fillna(merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXSCR'].min(), inplace=True)
merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXSGTSI'].fillna(merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXSGTSI'].min(), inplace=True)

In [22]:
def fill_na_for_Fasting_Blood_Sugar_value(row,field):
    if pd.isna(row[field]) and row['LBXGH'] < 5.7:
        row[field]  = (28.7 * row['LBXGH']) - 46.7
        if row[field] > 120:
            row[field] = 120
        return row[field] 
    elif pd.isna(row[field]) and row['LBXGH'] >= 5.7 and row['LBXGH'] < 6.4:
        row[field]  = (28.7 * row['LBXGH']) - 46.7
        if row[field] > 125:
            row[field] = 125
        return row[field] 
    elif pd.isna(row[field]) and row['LBXGH']  >= 6.4:
        row[field]  = (28.7 * row['LBXGH']) - 46.7
        return row[field] 
    return row[field]
    
merged_dfNHAENES2018_Cleaned_numeric_HBV['LBXGLU'] = merged_dfNHAENES2018_Cleaned_numeric_HBV.apply(fill_na_for_Fasting_Blood_Sugar_value, axis=1, field='LBXGLU')


In [23]:
print(merged_dfNHAENES2018_Cleaned_numeric_HBV)

      LBXGLU  LBXSGL  LBDHDD  LBDLDL  LBXTC  URXUCR  URDACT  LBXSCR  LBXSGTSI  \
0     125.00    85.0    60.0    18.0  157.0    24.0   13.33    0.92      21.0   
1     102.54    94.0    47.0    18.0  148.0    69.0   57.39    0.81      16.0   
2     114.02   115.0    68.0    18.0  189.0   209.0   19.90    0.64       9.0   
3     122.00   116.0    88.0   109.0  209.0    47.0   19.36    0.58      22.0   
4     125.00    96.0    65.0    18.0  176.0   215.0   12.28    1.32      31.0   
...      ...     ...     ...     ...    ...     ...     ...     ...       ...   
5946  158.00   150.0    60.0    43.0  119.0    51.0   10.39    0.70      18.0   
5947  122.63   101.0    49.0    18.0  182.0   231.0    4.42    0.92      28.0   
5948   94.00    88.0    54.0   108.0  172.0    31.0   10.97    0.69       8.0   
5949  111.15    84.0    34.0    18.0  150.0   177.0   47.01    0.85      16.0   
5950   91.00    90.0    34.0    75.0  163.0    75.0   70.93    0.82      72.0   

      LBXHGB  LBXMCHSI  LBX

In [24]:
nan_counts = merged_dfNHAENES2018_Cleaned_numeric_HBV.isna().sum()
print("\nCount of NaN values in each column:")
print(nan_counts)


Count of NaN values in each column:
LBXGLU      0
LBXSGL      0
LBDHDD      0
LBDLDL      0
LBXTC       0
URXUCR      0
URDACT      0
LBXSCR      0
LBXSGTSI    0
LBXHGB      0
LBXMCHSI    0
LBXMC       0
LBXIN       0
URXPREG     0
LBXTR       0
LBXSUA      0
BMXBMI      0
LBXGH       0
dtype: int64


In [25]:
merged_dfNHAENES2018_Cleaned_numeric_HBV=merged_dfNHAENES2018_Cleaned_numeric_HBV.drop('BMXBMI', axis=1)

In [26]:
merged_dfNHAENES2018_Cleaned_numeric_HBV.to_csv('Data\After_DataPreprocessing_Lab.csv', index=False)